# Toy example for Radix-based with nonzero forcing constraint

In [1]:
%load_ext line_profiler

In [2]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7fc4d0362d88>)


### Try optimizing using radix for one condition first

In [3]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound = 0.001
not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [4]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [5]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e3)

In [6]:
gap.optimize(solver='gurobi')

<Solution 1.75 at 0x7fc51424e950>

In [7]:
import numpy as np

radix = 2.
print('Radix:',radix)
powers = np.arange(-3,4)
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

('Radix:', 2.0)
('Powers:', array([-3, -2, -1,  0,  1,  2,  3]))
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [8]:
# %lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power, prevent_zero=True)

In [9]:
%time opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=True)

CPU times: user 133 ms, sys: 7.91 ms, total: 141 ms
Wall time: 149 ms


### DEBUG try relaxing non-zero constraint on groups found needing to be 0

[(u'PGK', 0.0), (u'GLUDy', 0.0), (u'PGM', 0.0), (u'PGI', 0.0), (u'RPI', 0.0)]

In [22]:
relaxed = ['PGK','GLUDy','PGM','PGI','RPI']
for rel in relaxed:
    cons = gap.metabolites.get_by_id('force_nonzero_%s'%rel)
    cons._bound = 0.

In [23]:
### Try scaling
crowding = gap.metabolites.crowding
bound0  = crowding._bound
f_scale = crowding._bound
crowding._bound = bound0 / f_scale
for rxn in crowding.reactions:
    rxn._metabolites[crowding] = rxn._metabolites[crowding]/f_scale

In [24]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [25]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [26]:
mu_meas = 0.74
#mu_meas = mu_crowd0

F_TOL = 0.01

gap.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = mu_meas*(1-F_TOL)
gap.reactions.BIOMASS_Ecoli_core_w_GAM.upper_bound = mu_meas*(1+F_TOL)
rxn_mu = gap.reactions.BIOMASS_Ecoli_core_w_GAM 

print('#'*40)
print('Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))
print('#'*40)

from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE
#milp.ModelSense = GRB.MAXIMIZE

milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.Params.FeasibilityTol = 1e-9
milp.Params.OptimalityTol = 1e-9
milp.optimize()

########################################
Initial mu_crowd=0.873922. Fitting within 1% of measured: 0.7326 <= mu <= 0.7474
########################################
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter FeasibilityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Changed value of parameter OptimalityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Optimize a model with 2905 rows, 1366 columns and 8292 nonzeros
Variable types: 1030 continuous, 336 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 1e+03]
  RHS range        [9e-01, 1e+03]
Presolve removed 884 rows and 204 columns
Presolve time: 0.05s
Presolved: 2021 rows, 1162 columns, 6136 nonzeros
Variable types: 840 continuous, 322 integer (322 binary)

Root relaxation: objective 4.300000e+01, 1364 iterations, 0.02 seconds

    Nodes    |    Curre

In [27]:
#gap.optimize('minimize')
rxn_mu =gap.reactions.BIOMASS_Ecoli_core_w_GAM 
#muopt = rxn_mu
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)
print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))
print('Number of non-zero binaries: %g' % sum(yopt))

Initial mu_crowd=0.873922. Fitted within 1%: 0.7326 <= 0.736021 <= 0.7474
Number of non-zero binaries: 43


In [34]:
len(var_cons_dict)

48

In [30]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
#                 print yid, y
#             if abs(y)<1e-10:
#                 print yid, y
                pass
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

In [31]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'G6PDH2r', 5.341880341880342e-07),
 (u'AKGDH', 8.547008547008548e-06),
 (u'ACKr', 3.418803418803419e-05),
 (u'MALS', 3.418803418803419e-05),
 (u'GLUN', 2.136752136752137e-06),
 (u'ME1', 3.418803418803419e-05),
 (u'GND', 4.273504273504274e-06),
 (u'ME2', 3.418803418803419e-05),
 (u'GLNS', 2.136752136752137e-06),
 (u'FUM', 1.7094017094017095e-05),
 (u'SUCDi', 4.273504273504274e-06),
 (u'PPC', 1.0683760683760685e-06),
 (u'MDH', 5.341880341880342e-07),
 (u'GLUSy', 2.136752136752137e-06),
 (u'PGL', 1.7094017094017095e-05),
 (u'ACALD', 3.4188034188034184e-05),
 (u'PPS', 5.341880341880342e-07),
 (u'PTAr', 3.418803418803419e-05),
 (u'TALA', 1.0683760683760685e-06),
 (u'ALCD2x', 1.0683760683760685e-06),
 (u'RPE', 2.136752136752137e-06),
 (u'NADTRHD', 5.341880341880342e-07),
 (u'ICDHyr', 2.136752136752137e-06),
 (u'GAPD', 1.7094017094017095e-05),
 (u'ICL', 3.418803418803419e-05),
 (u'TPI', 3.418803418803419e-05),
 (u'ENO', 1.7094017094017095e-05),
 (u'ACONTa', 5.341880341880342e-07),
 (u'SUCO

### Plug back in to be sure

In [32]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding_bound = ijomc.metabolites.crowding._bound
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

In [33]:
ijofit.optimize()
print('Mu=%g' % ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x)

Mu=0.736021
